In [1]:
# !pip install plotly finta matplotlib
!pip install ../ta -U


Processing c:\users\codeninja\dev\ta
  Created wheel for ta: filename=ta-0.4.7-cp37-none-any.whl size=18775 sha256=2a414af2bd4a53c84fd3648264d7f0fa42ddc0d58158177a918a26f8e08a8e10
  Stored in directory: C:\Users\CODENI~1\AppData\Local\Temp\pip-ephem-wheel-cache-2thzm6d2\wheels\71\d3\81\76a2b4c89b367a05e6c899b7dbeffa9e9a257871c466c4597e
Successfully built ta
  Found existing installation: ta 0.4.7
    Uninstalling ta-0.4.7:
      Successfully uninstalled ta-0.4.7


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
import os
from pathlib import Path
import sys
from ta import *
import sklearn

In [3]:
# Shared Methods 
# define a conversion function for the native timestamps in the csv file
def dateparse (time_in_miliseconds):
    return datetime.fromtimestamp(int(time_in_miliseconds)/1000)

Open the data file

In [4]:
print('Data listing...')
import os

# read in the data and apply our conversion function, this spits out a DataFrame with the DateTimeIndex already in place
data = pd.read_csv('../CCXT-Historical-Data/data/raw/binance/btc_usdt_1m.csv',
                   names = ['open','high','low','close','volume'],
                   dtype={'open': np.float64, 'high':np.float64, 'low':np.float64, 'volume':np.float64},
                   parse_dates=True,
                   date_parser=dateparse, 
                   index_col=[0])

print(data.columns)
print('Total null open prices: %s' % data['open'].isnull().sum())
print(data.head())
print(data.shape)

# Remove duplicated indexes and keep the first record
data = data.reset_index().drop_duplicates(subset='index', keep='first').set_index('index')


Data listing...
Index(['open', 'high', 'low', 'close', 'volume'], dtype='object')
Total null open prices: 0
                            open     high      low    close      volume
2018-02-09 03:59:14.789  7789.90  8230.46  7789.90  8230.46  148.475418
2018-02-09 04:00:14.789  8230.46  8317.74  8230.45  8317.74   91.494282
2018-02-09 04:01:14.789  8304.00  8369.00  8304.00  8350.00  131.872916
2018-02-09 04:02:14.789  8346.50  8380.00  8330.00  8380.00   84.544616
2018-02-09 04:03:14.789  8379.00  8390.00  8348.00  8360.02  112.940304
(902000, 5)


In [5]:
df = data.resample('1h').agg({
    'open': "first", 
    "high": 'max', 
    'low': 'min', 
    'close': 'last', 
    'volume': 'sum', 
    }).fillna(method='ffill')

In [6]:
print("Resample data into 1h timestamps")
df = data.resample('1h').agg({
    'open': "first", 
    "high": 'max', 
    'low': 'min', 
    'close': 'last', 
    'volume': 'sum', 
    }).fillna(method='ffill')

df2 = data.copy(deep=True)
# adx(df["high"], df["low"], df["close"], n=14, fillna='ffill')
# adx_neg(df["high"], df["low"], df["close"], n=14, fillna='ffill')
# adx_pos(df["high"], df["low"], df["close"], n=14, fillna='ffill')
df = add_all_ta_features(df, "open", "high", "low", "close", "volume", fillna='ffill')

Resample data into 1h timestamps
(15021, 5)


In [10]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'volume_adi', 'volume_obv',
       'volume_cmf', 'volume_fi', 'volume_em', 'volume_vpt', 'volume_nvi',
       'volatility_atr', 'volatility_bbh', 'volatility_bbl', 'volatility_bbm',
       'volatility_bbhi', 'volatility_bbli', 'volatility_kcc',
       'volatility_kch', 'volatility_kcl', 'volatility_kchi',
       'volatility_kcli', 'volatility_dch', 'volatility_dcl',
       'volatility_dchi', 'volatility_dcli', 'trend_macd', 'trend_macd_signal',
       'trend_macd_diff', 'trend_ema_fast', 'trend_ema_slow', 'trend_adx',
       'trend_adx_pos', 'trend_adx_neg', 'trend_vortex_ind_pos',
       'trend_vortex_ind_neg', 'trend_vortex_diff', 'trend_trix',
       'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst',
       'trend_kst_sig', 'trend_kst_diff', 'trend_ichimoku_a',
       'trend_ichimoku_b', 'trend_visual_ichimoku_a',
       'trend_visual_ichimoku_b', 'trend_aroon_up', 'trend_aroon_down',
       'trend_aroon_ind', 'momentum_rsi', 

**Cleanup Data & Add New Inputs**

We'll calculate the volume currency, and append indicator columns

In [12]:
print('Total null open prices: %s' % df.isnull().sum())
df.head()

Total null open prices: open             0
high             0
low              0
close            0
volume           0
                ..
momentum_ao      0
momentum_kama    0
others_dr        0
others_dlr       0
others_cr        0
Length: 64, dtype: int64


,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,...,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,others_dr,others_dlr,others_cr
index,,,,,,,,,,,,,,,,,,,,,
2018-02-09 03:00:00,7789.90,8230.46,7789.90,8230.46,148.475418,211.584960,0.000000,1.000000,0.000000,0.000000,...,100.000000,100.000000,100.000000,100.000000,-0.000000,0.0,8230.46,15.515512,0.000000,0.000000
2018-02-09 04:00:00,8230.46,8390.00,8166.00,8264.99,1984.058640,-81.994251,1984.058640,-0.038449,0.000000,30.236848,...,100.000000,90.597058,79.168472,89.584236,-20.831528,0.0,8264.99,0.419539,0.418662,0.419539
2018-02-09 05:00:00,8258.10,8395.00,8227.82,8288.03,1351.854008,-608.581246,3335.912648,-0.132048,69278.505426,17.184285,...,100.000000,84.500128,82.321930,87.163467,-17.678070,0.0,8288.03,0.278766,0.278378,0.699475
2018-02-09 06:00:00,8288.03,8300.00,8201.95,8241.00,1003.249556,-582.240375,2332.663092,-0.148014,23529.609925,9.487370,...,98.755190,81.753394,74.549661,78.680021,-25.450339,0.0,8241.00,-0.567445,-0.569061,0.128061
2018-02-09 07:00:00,8241.00,8340.00,8225.00,8300.00,1170.145293,152.002378,3502.808385,-0.054457,-2175.053319,7.890084,...,97.964632,80.641133,84.300116,80.390569,-15.699884,0.0,8300.00,0.715933,0.713382,0.844910


In [13]:
p = Path("./data/processed/binance/")
p.mkdir(parents=True, exist_ok=True)
full_path = p / 'btc_usdt_1h.csv'
    
df.to_csv(full_path, index=True)
print('Created', full_path)

Created data\processed\binance\btc_usdt_1h.csv
